<a href="https://colab.research.google.com/github/jellio9/Absorption_with_lvlspy/blob/main/Absorption_with_lvlspy_JacobElliott.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Absorption with lvlspy - Jacob Elliott


The goal of this notebook is to display the absorption spectrum from an optically thin gas of atoms. We will assume a flat spectrum for the incident radiation over a certain range, and we will look at the spectrum as a function of temperature of the gas.

Begin by installing lvlspy and other necessary libraries:

In [20]:
import sys, subprocess, importlib.util
required = {'numpy','lvlspy','matplotlib','scipy'}
installed = {pkg for pkg in required if importlib.util.find_spec(pkg) is not None}
missing = required - installed
if missing:
    subprocess.check_call([sys.executable, '-m','pip','install','--quiet',*missing])

import numpy as np
import lvlspy.level as lv
import lvlspy.spcoll as lc
import lvlspy.species as ls
import lvlspy.transition as lt
import lvlspy.io as li
from lvlspy.io import xml

from ipywidgets import interact, interactive
import ipywidgets as widgets
from scipy.integrate import odeint


!{sys.executable} -m pip install --quiet matplotlib

import matplotlib.pyplot as plt

# Create an xml file for lvlspy

Using the code from the levels.py documentation tutorial on create xml files, create the levels and print their energies and multiplicities.

In [21]:
levs = [lv.Level(0,1),lv.Level(100,1),lv.Level(200,4)]

s = ls.Species('test',levels=levs)
for lev in s.get_levels():
    print(lev.get_energy(), lev.get_multiplicity())

#constants
e = 4.8032 * 10**-10
h = 6.626 * 10 ** -27
c = 3 * 10 ** 10

0.0 1
100.0 1
200.0 4


Now add the transition levels and calculate the Einstein B lower-to-upper coefficients as well as the frequency (and hence wavelength).

In [22]:
t1_0 = lt.Transition(levs[1], levs[0], 100.)
t2_0 = lt.Transition(levs[2], levs[0], 100.)
t2_1 = lt.Transition(levs[2], levs[1], 100.)

print('Enstein B coefficient From Ground to First (per second): ',t1_0.get_einstein_b_lower_to_upper())
print('Enstein B coefficient From Ground to Second (per second): ',t2_0.get_einstein_b_lower_to_upper())
print('Enstein B coefficient From First to Second (per second): ',t2_1.get_einstein_b_lower_to_upper())

print('nu 1_0 (per second) =', t1_0.get_frequency())
print('nu 2_0 (per second) =', t2_0.get_frequency())
print('nu 2_1 (per second) =', t2_1.get_frequency())

#get wavelength in nanometers
l1_0 = (c / t1_0.get_frequency()) * 10 ** 7
l2_0 = (c / t2_0.get_frequency()) * 10 ** 7
l2_1 = (c / t2_1.get_frequency()) * 10 ** 7

print('lambda 1_0 (nm) =', l1_0)
print('lambda 2_0 (nm) =', l2_0)
print('lambda 2_1 (nm) =', l2_1)

Enstein B coefficient From Ground to First (per second):  4.797248893982106e-10
Enstein B coefficient From Ground to Second (per second):  2.398624446991053e-10
Enstein B coefficient From First to Second (per second):  1.9188995575928426e-09
nu 1_0 (per second) = 2.4179894544894685e+19
nu 2_0 (per second) = 4.835978908978937e+19
nu 2_1 (per second) = 2.4179894544894685e+19
lambda 1_0 (nm) = 0.012407002000897546
lambda 2_0 (nm) = 0.006203501000448773
lambda 2_1 (nm) = 0.012407002000897546


Add the Temperature dependence and transition probabilities.

In [23]:
T = 1.e9

print(t1_0.compute_lower_to_upper_rate(T))
print(t2_0.compute_lower_to_upper_rate(T))
print(t2_1.compute_lower_to_upper_rate(T))

# Compute and print out the equilibrium probabilities

p = s.compute_equilibrium_probabilities(T)

for i in range(len(p)):
    print('Level =', i, ', Probability =', p[i])

45.633537669992236
43.549989073174686
182.53415067996895
Level = 0 , Probability = 0.5861371715457715
Level = 1 , Probability = 0.18366313917421226
Level = 2 , Probability = 0.23019968928001636


Create the xml file.

In [24]:
my_coll = lc.SpColl([s])

for ss in my_coll.get():
    print(ss)

xml.write_to_xml(my_coll,'test.xml')

!cat test.xml

test
<species_collection>
  <species name="test">
    <levels>
      <level>
        <properties>
          <energy>0.0</energy>
          <multiplicity>1</multiplicity>
        </properties>
      </level>
      <level>
        <properties>
          <energy>100.0</energy>
          <multiplicity>1</multiplicity>
        </properties>
      </level>
      <level>
        <properties>
          <energy>200.0</energy>
          <multiplicity>4</multiplicity>
        </properties>
      </level>
    </levels>
  </species>
</species_collection>
